# Colab

In [ ]:
!git clone https://github.com/Tio-Panda/inf577-project.git

# Init

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from pathlib import Path

import tensorflow as tf
import keras


base_path = "." # local
#base_path = "/content/inf557-project" # colab

dataset_path = f"{base_path}/processed_dataset"
weights_path = f"{base_path}/weights"

Ns = 2800
Nc = 128
Nz = 2048
Nx = 256

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.keras.config.disable_traceback_filtering()
keras.backend.clear_session()
import gc; gc.collect()

# Load Data

In [ ]:
from modules import split_dataset

batch_size = 1
train, val = split_dataset(dataset_path, splits=(0.9, 0.1), batch_size=batch_size, buffer_size=batch_size, seed=11, Nc=Nc, Ns=Ns, Nz=Nz, Nx=Nx)

# Load Model

In [ ]:
from modules import get_model

k_m1 = [16, 8, -1, 8, 4, 1] # Model-1
k_m2 = [-1, 16, 8, 8, 4, 1] # Model-2
k_m3 = [16, 8, 8, 4, 1, -1] # Model-3

model = get_model(k_array=k_m1, version="FULL", Nc=Nc, Ns=Ns, Nz=Nz, Nx=Nx)
model.summary()

# Training

In [ ]:
model_name = "model-1.1"

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, min_delta=800,
    restore_best_weights=True, start_from_epoch=3
)

ckpt = keras.callbacks.ModelCheckpoint(
    filepath=weights_path+model_name+".keras",
    monitor="val_loss", save_best_only=True,
    save_weights_only=False
)

tensorboard = keras.callbacks.TensorBoard(log_dir="logs/"+model_name, profile_batch=0)

csv_logger = keras.callbacks.CSVLogger("./training_log/"+model_name+".csv", append=False)

callbacks = [ckpt, tensorboard, csv_logger]


hist = model.fit(train, validation_data=val, epochs=100, callbacks=callbacks)

# Get a reconstruction

In [ ]:
from modules import load_data, get_bmode, show_bmode

inputs, label = load_data("carotid_cross_long_dataset_rf", base_path)

custom_obj = {
    "LeakyReLU": keras.layers.LeakyReLU,
}
model1 = keras.saving.load_model(
    weights_path+model_name+".keras",
    custom_objects=custom_obj
)

In [ ]:
from modules import get_bmode, show_bmode

img = model1.predict(inputs)

vmin = -45
bimg = get_bmode(img[0, ..., 0])
show_bmode(bimg, tf.squeeze(inputs["grid"]), vmin=vmin)

bimg = get_bmode(label)
show_bmode(bimg, tf.squeeze(inputs["grid"]), vmin=vmin)